In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
submission.head()

In [ ]:
train_df.info()

In [ ]:
train_df.corr()['SalePrice']

In [ ]:
train_df.describe()

In [ ]:
new_col = list(train_df.corr()['SalePrice'][train_df.corr()['SalePrice']>0.50].index)

print(new_col)

In [ ]:
df = train_df[new_col]
df.head()

In [ ]:
df.isna().sum()

In [ ]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_x = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(scaled_x, y, test_size = 0.17, random_state = 42)

In [ ]:
from sklearn.linear_model import LinearRegression

lir = LinearRegression()

lir = lir.fit(x_train, y_train)

print(lir.score(x_test, y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rgb = RandomForestRegressor()

rgb = rgb.fit(x_train, y_train)

print(rgb.score(x_test, y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

hyper_params = {
    'max_depth': [3,4,5,6,7,8,9],
    'learning_rate': [0, 0.5, 0.001, 0.1, 0.2],
    'gamma':[0, 0.5,1, 1.5, 2, 5],
    'subsample':[0.6, 0.7, 0.8, 0.9, 1],
    'colsample_bytree':[0.6, 0.7, 0.8, 0.9, 1],
    'max_bin':[256, 512, 1024]
}

xgb = XGBRegressor(seed=13, tree_method = 'hist')

rcv = RandomizedSearchCV(
    estimator = xgb,
    param_distributions = hyper_params,
    n_iter = 4,
    scoring = 'neg_root_mean_squared_error',
    verbose = 3
)

rcv.fit(x_train, y_train)

print(rcv.best_params_)
print(rcv.best_score_)

In [ ]:
xgb = XGBRegressor(
    seed= 13,
    tree_method = 'hist',
    subsample = 0.8,
    max_depth = 7,
    max_bin = 256,
    learning_rate = 0.1,
    gamma = 0,
    colsample_bytree = 1,
)

xgb = xgb.fit(
    x_train, y_train,
    eval_metric = "rmse"
)

In [ ]:
print(xgb.score(x_test, y_test))

In [ ]:
test_df.info()

In [ ]:
df = test_df[new_col[:-1:]]
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df['TotalBsmtSF'].fillna(df.describe()['TotalBsmtSF']['50%'])
df['GarageCars'].fillna(df.describe()['GarageCars']['50%'])
df['GarageArea'].fillna(df.describe()['GarageArea']['50%'])

In [ ]:
df.isna().sum()

In [ ]:
scaled_x = scaler.fit_transform(df)

In [ ]:
y_pred_xgb = xgb.predict(scaled_x)

In [ ]:
submission.head()

In [ ]:
output = pd.DataFrame({"Id":test_df['Id'],'SalePrice':y_pred_xgb})
output.to_csv('submission.csv', index = False)